In [1]:
from datasets import load_dataset
import json
import nltk
from nltk.corpus import stopwords
import re

In [2]:
dataset = load_dataset('poloclub/diffusiondb', '2m_first_5k')

Found cached dataset diffusiondb (C:/Users/46763/.cache/huggingface/datasets/poloclub___diffusiondb/2m_first_5k/0.9.1/547894e3a57aa647ead68c9faf148324098f47f2bc1ab6705d670721de9d89d1)
100%|██████████| 1/1 [00:00<00:00,  7.25it/s]


In [3]:
num_rows = dataset['train'].num_rows

In [4]:
stop_words = set(stopwords.words('english'))
min_word_length = 2
min_word_freq = 5

In [5]:
custom_word_map = {}
for i in range(num_rows):
    prompt = dataset['train']['prompt'][i]
    clean_prompt = re.sub('[^A-Za-z]+',' ', prompt)
    clean_prompt_arr = clean_prompt.split(' ')
    for word in clean_prompt_arr:
        if len(word) >= min_word_length and word not in stop_words:
            if word in custom_word_map.keys():
                custom_word_map[word] += 1
            else: 
                custom_word_map[word] = 1

In [6]:
with open('WORDMAP_coco_5_cap_per_img_5_min_word_freq.json') as json_file:
    existing_word_map = json.load(json_file)

In [7]:
clean_existing_word_map = {}
for key, value in existing_word_map.items():
    clean_word = re.sub('[^A-Za-z]+',' ', key)
    if len(clean_word) >= min_word_length and clean_word not in stop_words:
        clean_existing_word_map[clean_word] = value

In [8]:
special_tokens = {
    "<pad>": 0,
}
unk_token = {
    "<start>": 1,
    "<end>": 2,
    "<unk>": 3,
}

In [9]:
final_word_map = special_tokens | clean_existing_word_map | custom_word_map | unk_token

In [10]:
final_word_map

{'<pad>': 0,
 '<start>': 1,
 '<end>': 2,
 'man': 153,
 'with': 3,
 'red': 132,
 'helmet': 25,
 'on': 6,
 'small': 230,
 'moped': 8,
 'dirt': 3,
 'road': 11,
 'riding': 12,
 'motor': 12,
 'bike': 13,
 'the': 14,
 'countryside': 1,
 'back': 50,
 'of': 17,
 'motorcycle': 18,
 'path': 13,
 'young': 65,
 'person': 24,
 'rests': 22,
 'to': 23,
 'foreground': 2,
 'verdant': 25,
 'area': 2,
 'bridge': 7,
 'and': 28,
 'background': 251,
 'cloud': 9,
 'mountains': 10,
 'in': 32,
 'shirt': 10,
 'hat': 53,
 'is': 35,
 'hill': 14,
 'side': 28,
 'woman': 299,
 'wearing': 408,
 'net': 40,
 'her': 41,
 'head': 131,
 'cutting': 43,
 'cake': 10,
 'large': 252,
 'white': 268,
 'sheet': 1,
 'hair': 175,
 'there': 49,
 'that': 50,
 'marking': 51,
 'chefs': 52,
 'knife': 1,
 'child': 4,
 'holding': 49,
 'flowered': 56,
 'umbrella': 2,
 'petting': 58,
 'yak': 59,
 'an': 60,
 'next': 11,
 'herd': 62,
 'cattle': 63,
 'boy': 17,
 'barefoot': 65,
 'touching': 2,
 'horn': 67,
 'cow': 4,
 'who': 69,
 'while': 70,


In [11]:
for i, (key, value) in enumerate(final_word_map.items()):
    final_word_map[key] = i

In [12]:
print("Final length of word map:", len(final_word_map))

Final length of word map: 13925


In [13]:
with open('word_map_nostop.json', 'w') as fp:
    json.dump(final_word_map, fp)